In [1]:
import numpy as np
import cv2
import sklearn
import pickle
import copy

In [2]:
#  same- trained models used to detect the face from the test images
face_detector_model = cv2.dnn.readNetFromCaffe('./models/deploy.prototxt.txt',
                                               './models/res10_300x300_ssd_iter_140000.caffemodel')
# feature extraction
face_feature_model = cv2.dnn.readNetFromTorch('./models/openface.nn4.small2.v1.t7')
# emotion recognition model
emotion_recognition_model = pickle.load(open('./models/emotion_detection.pkl',mode='rb'))

In [ ]:
#  image to test the results from the test folder
img = cv2.imread('./test/test1.jpg')
image = img.copy()
h,w = img.shape[:2]
# face detection
img_blob = cv2.dnn.blobFromImage(img,1,(300,300),(104,177,123),swapRB=False,crop=False)
face_detector_model.setInput(img_blob)
detections = face_detector_model.forward()
if len(detections) > 0:
    for i, confidence in enumerate(detections[0, 0, :, 2]):
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            startx, starty, endx, endy = box.astype(int)
            cv2.rectangle(img,(startx, starty), (endx, endy), (0, 255, 0))


            # feature extraction
            face_roi = img[starty:endy, startx:endx]
            face_blob = cv2.dnn.blobFromImage(face_roi, 1 / 255, (96, 96), (0, 0, 0), swapRB=True, crop=True)
            face_feature_model.setInput(face_blob)
            vectors = face_feature_model.forward()
            # EMOTION
            emotion_name = emotion_recognition_model.predict(vectors)[0]
            emotion_score = emotion_recognition_model.predict_proba(vectors).max()*100
            emotion_score_percent = "{:.2f}".format(emotion_score)
#             printing the name of the emotion with score
            print(emotion_name)
            print(emotion_score_percent +"%")
#             display the box around on the detected face on the image with results
            text_emotion = '{} : {:.0f} %'.format(emotion_name, 100 * emotion_score)
            cv2.putText(img, text_emotion, (startx, endy), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2)





cv2.imshow('Emotion_Detection',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

happy
30.79%
angry
25.21%
neutral
34.62%


## Conclusion
## this project sucessfully detected the face from the images and also successfuly detected the emotions around the faces.
## the accuracy was very low for the project, but this accuracy was expected because of the small dataset. more accuracy can be achieved by adding more images with emotions in the dataset. which required more space and power in the machine. this project was still accurately achieved the results from the test image image.